# * Ex-T Revenue & Subs

## Parameter

In [50]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Ex-T Revenue
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [49]:
''' Execute query '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ REMARK, METRIC_CD, METRIC_NAME
        , SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
        , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
        , CAST(SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) AS DECIMAL(18)) C
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) AS DECIMAL(18)) P
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) AS DECIMAL(18)) G
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) AS DECIMAL(18)) H
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) AS DECIMAL(18)) HH
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) AS DECIMAL(18)) CCAA
        , CAST(SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) AS DECIMAL(18)) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    WHERE REMARK IN ('salisa', 'jutar11')
    AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
    --ORDER BY REMARK, METRIC_CD, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Ex-T Revenue
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Ex-T Revenue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-06, 18:27:33

   -> Ex-T Revenue DataFrame : 418 rows, 17 columns

TDMDBPR : Disconnected


In [81]:
''' METRIC Summary '''

src_t_revenue_df = chk_src_df.copy()
src_t_revenue_df = src_t_revenue_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_revenue_df = src_t_revenue_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_revenue_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_revenue_df[col] = src_t_revenue_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_revenue_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-05 04:53:19   20240101  20250302                  0              5         1104   471408
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-05 04:49:11   20240101  20250302                  0              4          176    75152
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-05 04:51:34   20240101  20250302                  0              4          176    75152
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-05 04:53:25   20240101  20250303                  0              5         1104   471408
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-05 04:53:27   20240101  20250303      6,174,691,544              6         1105   472927
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-05 04:21:17   20240102  

In [84]:
''' METRIC by Month '''

v_tm_key_mth = '202502'
src_t_revenue_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

src_t_revenue_df = chk_src_df[src_t_revenue_df_cols].copy()
src_t_revenue_df = src_t_revenue_df.loc[src_t_revenue_df['TM_KEY_MTH']==v_tm_key_mth]
src_t_revenue_df = src_t_revenue_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_revenue_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_revenue_df[col] = src_t_revenue_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_revenue_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202502

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C              P              G              H             HH           CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-05 04:47:05     202502   20250201  20250228  0              0  2,273,575,220  2,273,562,664  2,273,562,664  2,264,108,728      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-05 04:42:38     202502   20250201  20250228  0              0    254,667,955    254,667,955    254,667,955              0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-05 04:45:50     202502   20250201  20250228  0              0  2,018,907,264  2,018,894,708  2,018,894,708              0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-03 04:50:27     202502   20250201  20250228  0              0    492,224,701    492,224,701    492,224,701    490,496,98